In [1]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [2]:
#Save the dataset as a DataFrame, and print the schema.
twitter = sqlContext.read.json("file:///home/cloudera/Desktop/Assignment/tweets.json")
twitter.printSchema()

root
 |-- country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- place: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user: string (nullable = true)



In [3]:
#Save the dataset as a DataFrame, and print the schema.

twitter.registerTempTable("twitter")
twitter.cache()

sqlContext.sql("SELECT count(1) as tweets FROM twitter").show()

+------+
|tweets|
+------+
|  8198|
+------+



In [5]:
#Get all of the tweets made by a user (any user would work. We should
#be able to replace user names to get tweets by that particular user).

username = 'stefan'

users = sqlContext.sql('SELECT text,user, country,id FROM twitter WHERE user ="{}"'.format(username)).show()


+--------------------+------+--------------+------------------+
|                text|  user|       country|                id|
+--------------------+------+--------------+------------------+
|@traceyb65 I'm up...|stefan|United Kingdom|572667477949353984|
+--------------------+------+--------------+------------------+



In [7]:
#Find count of all tweets by each user user.

sqlContext.sql("SELECT user, count(1) as tweets_Byuser FROM twitter GROUP BY user").show(10)


+-------------------+-------------+
|               user|tweets_Byuser|
+-------------------+-------------+
|             вяεиdα|            1|
|        Natty Dread|            1|
|          Cat Woman|            1|
|     goggle chroome|            1|
|      s a v a n n a|            1|
|milena visnieski ☯ |            1|
|     Virgie Riddick|            1|
|      BIG COUNTRY J|            1|
|          Ray Heard|            1|
|       Raven Givens|            1|
+-------------------+-------------+
only showing top 10 rows



In [11]:
#Get a list of all of the people who are mentioned in tweets.
from pyspark.sql.functions import split, explode, col

mentions =sqlContext.sql("SELECT text from twitter where text like '@%'")


allpeoplementioned=mentions.select(explode(split("text", "\\s+")).alias("word")).where(col("word").startswith("@"))

allpeoplementioned.show()








+----------------+
|            word|
+----------------+
|   @always_nidhi|
|        @YouTube|
|    @OnlyDancers|
|       @BeezyDH_|
|   @blakeshelton|
|   @DjGregStreet|
|      @traceyb65|
|      @hokkazonn|
|        @daunugh|
|         @3Will1|
|  @4fucksakesmag|
|     @Ashton5SOS|
|   @jamiisonkent|
|      @Jazzfeezy|
|    @NBCTheVoice|
|  @AaliyahSymone|
|    @MR_JAYJONES|
|@Gurmeetramrahim|
|  @TheDurttyBoyz|
|    @MR_JAYJONES|
+----------------+
only showing top 20 rows



In [12]:
#Count the number of time each person is mentioned in the entire
#dataset of tweets.

allpeoplementioned.groupBy("word").count().show(50)

+----------------+-----+
|            word|count|
+----------------+-----+
|     @_LOWEN_III|    1|
|         @bdotTM|    1|
|@The_Angel_Perez|    1|
|   @NeYoCompound|    1|
| @BI_performance|    1|
|  @shiranproject|    1|
|     @wakachachi|    1|
| @IsaacGarfinkle|    1|
|   @S_Adlington.|    1|
|       @KatiaC72|    1|
|     @infomanado|    1|
| @AdmiringJuliaB|    1|
|       @RepCohen|    1|
| @TheRealQueenJo|    1|
|    @hisbossness|    1|
|      @marIboros|    1|
|    @ShawnMendes|  187|
|@officialdjjuice|   55|
|       @TP_Grant|    1|
|     @Sonicfan71|    1|
|      @robpursey|    1|
|     @MarketsNYC|    1|
|  @bretteldredge|    2|
| @carterreynolds|    1|
|   @dymond_Tyche|    1|
|        @BigSean|    1|
|     @emme_white|    1|
|   @Kaylaa_Grace|    1|
|  @DILLONFRANCIS|    1|
|@xyougetsoalonex|    1|
|    @CiCis_Pizza|    1|
|         @spinph|    1|
|@TonyaBoydCannon|    1|
|    @Kidd_Suavay|    1|
| @RichardBrancat|    1|
|         @Foodie|    1|
|   @TheEllenShow|    1|


In [84]:
#Give top 50 users who are mentioned the most.

allpeoplementioned.groupBy("word").count().orderBy("count", ascending=False).take(50)

[Row(word=u'@ShawnMendes', count=187),
 Row(word=u'@HIITMANonDECK', count=95),
 Row(word=u'@officialdjjuice', count=55),
 Row(word=u'@MR_JAYJONES', count=39),
 Row(word=u'@MadisonElleBeer', count=25),
 Row(word=u'@NICKIMINAJ', count=25),
 Row(word=u'@MeekMill', count=25),
 Row(word=u'@ShadowElijahUK', count=17),
 Row(word=u'@ODTDANK', count=17),
 Row(word=u'@waveyboyjam', count=17),
 Row(word=u'@ParisHilton', count=10),
 Row(word=u'@JackJackJohnson', count=7),
 Row(word=u'@ceejayseniour', count=7),
 Row(word=u'@PapychuroBwege', count=7),
 Row(word=u'@', count=6),
 Row(word=u'@taylorswift13', count=6),
 Row(word=u'@waziridello1', count=6),
 Row(word=u'@KeithUrban', count=6),
 Row(word=u'@justinbieber', count=5),
 Row(word=u'@DulyMisuly2', count=5),
 Row(word=u'@Mathiasedson1', count=5),
 Row(word=u'@ArianaGrande', count=4),
 Row(word=u'@DJADMAFUVU', count=4),
 Row(word=u'@BDozen', count=4),
 Row(word=u'@LOADMusicCanada', count=4),
 Row(word=u'@AnnaPeterTz', count=4),
 Row(word=u'@jackgi

In [15]:
#Get a list of all hashtags mentioned in the dataset.
from pyspark.sql.functions import split, explode, col

hashtags =sqlContext.sql("SELECT text from twitter where text like '#%'")
allhashmentioned=hashtags.select(explode(split("text", "\\s+")).alias("word")).where(col("word").startswith("#"))
allhashmentioned.collect()




[Row(word=u'#CurrentlyVibingTo'),
 Row(word=u'#MusicMonday'),
 Row(word=u'#VVIPCulture'),
 Row(word=u'#MONDAYNIGHTTEASE'),
 Row(word=u'#PULLUP'),
 Row(word=u'#music'),
 Row(word=u'#fun'),
 Row(word=u'#Disneyland'),
 Row(word=u'#Paris'),
 Row(word=u'#Passing'),
 Row(word=u'#cafe'),
 Row(word=u'#Paris'),
 Row(word=u"#TurnOffTheLights...Today's"),
 Row(word=u'#Q971'),
 Row(word=u'#Arianators'),
 Row(word=u'#BestFanArmy'),
 Row(word=u'#iHeartAwards'),
 Row(word=u'#myxmusicawards'),
 Row(word=u'#TAMUFollowTrain'),
 Row(word=u'#OurGreatLifeboat'),
 Row(word=u'#music'),
 Row(word=u'#music'),
 Row(word=u'#music'),
 Row(word=u'#internasionalfieldstudy#paris'),
 Row(word=u'#ShakedownRadio'),
 Row(word=u'#Podcast'),
 Row(word=u'#Sydney'),
 Row(word=u'#Australia'),
 Row(word=u'#Radio'),
 Row(word=u'#Music'),
 Row(word=u'#WakeUp'),
 Row(word=u'#BelleJourn\xe9e'),
 Row(word=u'#Paris'),
 Row(word=u'#radar'),
 Row(word=u'#autoroute\u2026'),
 Row(word=u'#arnaque'),
 Row(word=u'#tunnel'),
 Row(word=u'#A

In [16]:
#Find how many times each hashtag is mentioned in the dataset.

allhashmentioned.groupBy("word").count().show(50)




+--------------------+-----+
|                word|count|
+--------------------+-----+
|         #CEBCareers|    1|
|             #TVF420|    1|
|            #Webinar|    2|
|           #industry|    1|
|                 #dj|    1|
|          #MoneyGxng|    1|
|          #notredame|    1|
|        #TweetMyJobs|    2|
|         #motherhood|    1|
|          #goodnight|    1|
|             #PULLUP|    2|
|            #Podcast|    1|
|             #tunnel|    1|
|         #MonteCarlo|    1|
|       #AllInClusive|    1|
|               #Zedd|    2|
|             #Sydney|    1|
|    #DeLunesADomingo|    1|
|          #Jerusalem|    1|
|        #DemiPension|    2|
|          #SAXOPHONE|    1|
|                 #Dj|    1|
|#VedicHoliHealthy...|    1|
|             #Voyage|    3|
|               #boys|    1|
|              #COP21|    1|
|            #dvmusis|    1|
|  #CurrentlyVibingTo|    1|
|  #ButILoveHerToDeaf|    1|
|             #taptap|    1|
|               #Tour|    1|
|             

In [91]:
#Get a list of all of the people who are located in a particular city

place ='Paris'
sqlContext.sql('SELECT user,place FROM twitter where place  ="{}"'.format(place)).show()



+--------------------+-----+
|                user|place|
+--------------------+-----+
|     Trendsmap Paris|Paris|
|            Codeclic|Paris|
|        EPHYRE Paris|Paris|
|  Helene PASQUALETTI|Paris|
|     Takaaki Kishida|Paris|
|    Michael S. Osman|Paris|
|     Marie-Charlotte|Paris|
|    Juliette Laridan|Paris|
| Boscheron Etiopathe|Paris|
|                 たいき|Paris|
|   Maxence d'Aubigny|Paris|
|        Lionel Rigal|Paris|
|         Foued KEFIF|Paris|
|                 Wil|Paris|
|         FranceBidet|Paris|
|     Trendsmap Paris|Paris|
|  مطر بن نومه الكتبي|Paris|
|    Russell Williams|Paris|
|          Lauralou B|Paris|
| ART aujourd'hui gal|Paris|
+--------------------+-----+
only showing top 20 rows



In [21]:
#Get country wise distribution of users, and find out which country
#ranks highest in terms of number of tweets, and number of users.
countrywise = sqlContext.sql("SELECT count(user) as user_count,country FROM twitter group By country")

In [22]:
countrywise.orderBy("user_count", ascending=False).show()

+----------+--------------------+
|user_count|             country|
+----------+--------------------+
|      4841|       United States|
|       737|              France|
|       370|           Indonesia|
|       365|      United Kingdom|
|       256|              Brasil|
|       172|              Canada|
|       151|Republika ng Pili...|
|       104|           Argentina|
|        92|        South Africa|
|        90|           Australia|
|        66|               India|
|        59|              México|
|        57|                  日本|
|        53|              España|
|        50|            Malaysia|
|        44|               Kenya|
|        42|             Türkiye|
|        38|               Ghana|
|        33|            Colombia|
|        33|             Nigeria|
+----------+--------------------+
only showing top 20 rows



In [30]:
#Find out number of tweets where a user is from France and mentions
#Paris in their tweets.


place ='France'
placeRegion =sqlContext.sql('SELECT text FROM twitter where country  ="{}"'.format(place))

placeRegion = placeRegion.select(explode(split("text", "\\s+")).alias("word")).where(col("word").startswith("Paris"))

placeRegion.collect()

placeRegion.count()

299